In [3]:
import AxisArrays: AxisArray

In [80]:
data = reshape(1:8, 2, 4)

V = AxisArray(data; row=[:a, :b], col=1:4)

2-dimensional AxisArray{Int64,2,...} with axes:
    :row, [:a, :b]
    :col, 1:4
And data, a 2×4 reshape(::UnitRange{Int64}, 2, 4) with eltype Int64:
 1  3  5  7
 2  4  6  8

In [78]:
V[row=:a, col=1:3].axes

(AxisArrays.Axis{:col, UnitRange{Int64}}(1:3),)

In [64]:
using Distributions

y = rand(Bernoulli(0.7), 100)

100-element Vector{Bool}:
 0
 0
 1
 0
 1
 1
 0
 0
 0
 1
 1
 1
 0
 ⋮
 1
 0
 1
 0
 1
 1
 1
 0
 0
 1
 0
 1

In [65]:
using Turing, Distributions, StatsPlots, DistributionsAD

@model function coinflip(y)
    p ~ Beta(1, 1)
    
    y ~ filldist(Bernoulli(p), length(y))
end

chain = sample(coinflip(y), NUTS(), 10000)

┌ Info: Found initial step size
└   ϵ = 0.8
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (10000×15×1 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 0.5 seconds
Compute duration  = 0.5 seconds
parameters        = p
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Use `describe(chains)` for summary statistics and quantiles.


In [66]:
using MCMCChains

describe(chain)

Chains MCMC chain (10000×15×1 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 0.5 seconds
Compute duration  = 0.5 seconds
parameters        = p
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Summary Statistics
  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ess_per_sec 
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64       Float64 

           p    0.5681    0.0484    0.0008   4192.6871   6397.3049    1.0000     8435.9902

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

           p    0.4713    0.5356    0.5687    0.6022    0.6606


In [67]:
fieldnames(typeof(chain))

(:value, :logevidence, :name_map, :info)

In [68]:
post_p = chain.value[iter=:, var=:p, chain=1].data

10000-element Vector{Float64}:
 0.5649309605385094
 0.5585119932070854
 0.49674874398015706
 0.581541235203961
 0.6073683128806192
 0.5904421138766905
 0.5904421138766905
 0.5671239025622562
 0.5642034341743951
 0.6413754869918873
 0.6413754869918873
 0.6287786602511314
 0.571269522069502
 ⋮
 0.6575514232174947
 0.5807783690095322
 0.614400677726784
 0.6272245899040262
 0.6155417091026344
 0.5654477190286757
 0.6014931429544349
 0.5812369549870169
 0.5432540286529595
 0.5736121388077747
 0.4927698080691437
 0.455481390333534

In [69]:
mean(post_p)

0.5680577678045026

In [70]:
mean(y)

0.57

In [71]:
function compute_loglikelihood(y, post_p, log_likelihood)
    mean_post_p = mean(post_p)
    d_hat = sum(log_likelihood.(y, mean_post_p))

    @info "the value of d_hat: $d_hat"
    d = 0
    for p_ins in post_p
        d += sum(log_likelihood.(y, p_ins))
    end

    d /= length(post_p)
    return d_hat - d
end


compute_loglikelihood(y, post_p, (r, p) -> logpdf(Bernoulli(p), r))

[ Info: the value of d_hat: -68.33226033074283


0.48391143302477246

In [72]:
using StatsBase: countmap

countmap(y)

Dict{Bool, Int64} with 2 entries:
  0 => 43
  1 => 57

In [75]:
rand(Normal(0, 1), 100)

100-element Vector{Float64}:
  0.8273290717574224
 -0.33879503050879417
 -0.879940879907407
  0.6158374003723452
  2.7225721280432023
  0.3028734719020684
 -0.8425386042649556
 -0.04255565912173277
  1.2152194681367772
 -1.9100682947708725
 -0.07185486052385631
  0.3375068142130112
  1.3189591434994246
  ⋮
  0.022984542862224146
  1.6065244086619512
  0.8152320574573139
  0.9544226050437927
  1.5638569886516904
  0.08962001192988943
 -0.5219167001909587
 -0.2352060253939655
 -2.5152809087872963
  1.2579115224824917
 -0.6110089746887154
  0.350230301970348

In [76]:
supertype(typeof(Normal(0, 1)))

Distribution{Univariate, Continuous}

In [83]:
using ArviZ

hdi(post_p; prob=0.95)

(lower = 0.46939837883871466, upper = 0.6584173019191294)